# Zero Flowrate Degenerate Example

with three separation units

Created by Alex Dowling (alexdowling.net) at the University of Wisconsin-Madison

Load modeling package (JuMP) and optimization solver (IPOPT)

In [1]:
using JuMP
using Ipopt

# Define Problem

Declare streams, components (chemical species), and unites

In [2]:
# Streams
S = 2:9

# Components (chemical species)
C = ["A", "B"]

# Units
U = ["U1", "U2", "U3"]

3-element Array{ASCIIString,1}:
 "U1"
 "U2"
 "U3"

Specify flowrate connectivity using dictionaries

In [3]:
inlets = Dict{ASCIIString,Integer}("U1"=>2, "U2"=>5, "U3"=>4)
outletV = Dict{ASCIIString,Integer}("U1"=>3, "U2"=>6, "U3"=>8)
outletL = Dict{ASCIIString,Integer}("U1"=>4, "U2"=>7, "U3"=>9)

Dict{ASCIIString,Integer} with 3 entries:
  "U3" => 9
  "U1" => 4
  "U2" => 7

Specify feed streams and total feed flowrate

In [4]:
feeds = [2, 5]
feedflow = Dict{AbstractString, Float64}("A"=>0.55, "B"=>0.45)

Dict{AbstractString,Float64} with 2 entries:
  "B" => 0.45
  "A" => 0.55

Define costs for operating each unit

In [5]:
ecost = Dict{AbstractString,Float64}("U1"=>1.5, "U2"=>1.0)

Dict{AbstractString,Float64} with 2 entries:
  "U1" => 1.5
  "U2" => 1.0

Specify equilibrium partition coefficient for each unit

In [6]:
K = Dict{Tuple{ASCIIString, ASCIIString}, Float64}(
		("U1","A")=>1.008,		("U1","B")=>0.9,
		("U2","A")=>1.099,		("U2","B")=>0.9,
		("U3","A")=>1.093,		("U3","B")=>0.9	)

Dict{Tuple{ASCIIString,ASCIIString},Float64} with 6 entries:
  ("U3","A") => 1.093
  ("U1","B") => 0.9
  ("U2","B") => 0.9
  ("U1","A") => 1.008
  ("U3","B") => 0.9
  ("U2","A") => 1.099

# Build Optimization Model

In [7]:
m = Model(solver=IpoptSolver())

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is Ipopt

## General flowsheet model

In [8]:
# Total molar flowrate
@variable(m, f[S] >= 0)

# Component molar flowrate
@variable(m, fc[S,C] >= 0)

# Component mole fraction
@variable(m, 0.001 <= x[S,C] <= 1)

# Unit model
for u in U
	
	i = inlets[u]
	v = outletV[u]
	l = outletL[u]
	
	# Overall mass balance
	@constraint(m, f[i] == f[v] + f[l])

	for c in C
		# Component mass balance
		@constraint(m, fc[i,c] == fc[v,c] + fc[l,c])
	
		# vapor-liquid equilibrium
		@constraint(m, x[v, c] == K[(u,c)]*x[l,c])
	
	end
	
	# Unit summation (Rachford-Rice equation)
	# This constraint is redundant!!!
	# @constraint(m, sum{x[v,c] - x[l,c], c in C} == 0)
	
end

# Stream model: mole fraction specification
for s in S
	for c in C
		@constraint(m, fc[s,c] == f[s]*x[s,c])
	end
end

## Additional specifications

In [9]:
@variable(m, 0.551 <= purityA <= 1.0)
@variable(m, 0.9 <= recoveryA <= 1.0)

# Equipment cost
ecost = Dict{AbstractString,Float64}("U1"=>1.5, "U2"=>1.0, "U3"=>0.5)

# Set total feed to one
@constraint(m, sum{f[s], s in feeds} == 1)

# Calculate purity of A
@constraint(m, purityA*sum{sum{f[s], s in outletV[u]}, u in U} == sum{sum{fc[s,"A"], s in outletV[u]}, u in U})

# Calculate recovery of A
@constraint(m, recoveryA == sum{sum{fc[s,"A"], s in outletV[u]}, u in U} / feedflow["A"])

# Objective
@objective(m, Min, sum{sum{f[s], s in inlets[u]}*ecost[u], u in U} - 100*purityA)

for s in feeds
	for c in C
		setupperbound(x[s,c], feedflow[c])
		setlowerbound(x[s,c], feedflow[c])
	end
end

## Initialization

Strategy: Set all stream total molar flowrates to unity and assume composition matches feed

In [10]:
for s in S
	
	setvalue(f[s], 1.0)
	
	for c in C
		setvalue(fc[s,c], feedflow[c])
		setvalue(x[s,c], feedflow[c])
	end
end

## Print Model

In [11]:
m

Minimization problem with:
 * 17 linear constraints
 * 17 quadratic constraints
 * 42 variables
Solver is Ipopt

# Solve Model

In [12]:
solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:       98
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       15

Total number of variables............................:       38
                     variables with only lower bounds:       24
                variables with lower and upper bounds:       14
                     variables with only upper bounds:        0
Total number of equality constraints.................:       34
Total number of inequality constraint

:Optimal

Examine solution

In [13]:
for s in S
    print("Stream ",s,"\t")
    @printf "f = %.4f \t" getvalue(f[s])
    for c in C
        @printf "x_%s = %.4f \t" c getvalue(x[s,c]) 
    end
    println(" ")
end

Stream 2	f = 0.0000 	x_A = 0.5500 	x_B = 0.4500 	 
Stream 3	f = 0.0000 	x_A = 0.5025 	x_B = 0.4729 	 
Stream 4	f = 0.0000 	x_A = 0.4985 	x_B = 0.5254 	 
Stream 5	f = 1.0000 	x_A = 0.5500 	x_B = 0.4500 	 
Stream 6	f = 0.8945 	x_A = 0.5553 	x_B = 0.4448 	 
Stream 7	f = 0.1055 	x_A = 0.5053 	x_B = 0.4942 	 
Stream 8	f = 0.0000 	x_A = 0.5217 	x_B = 0.4729 	 
Stream 9	f = 0.0000 	x_A = 0.4773 	x_B = 0.5254 	 


# Run Degeneracy Hunter

In [14]:
include("../DegeneracyHunter.jl")

DegeneracyHunter

In [15]:
DegeneracyHunter.degeneracyHunter(m);

******************************************
Welcome to Degeneracy Hunter!
 
Options: 
includeBounds = false
includeWeaklyActive = true
removeFixedVar = true
epsiActive = 1.0e-6
epsiLambda = 1.0e-6
lambdaM = 100000.0
 
Setting up NLP evaluator... 1.1467e-5 seconds
Initializing... 1.266733188 seconds
Determining Jacobian structure... 0.023807954 seconds
Evaluating Jacobian... 0.030439849 seconds
 
Smallest non-zero element in Jacobian = -9.829267948159997e-41
Variable: 
x[27] = x[3,A]
Equation: 
-f[3]*x[3,A] + fc[3,A] = 0
 
Largest non-zero element in Jacobian = -1.8181818181818181
Variable: 
x[11] = fc[3,A]
Equation: 
recoveryA - 1.8181818181818181 fc[3,A] - 1.8181818181818181 fc[6,A] - 1.8181818181818181 fc[8,A] = 0
 
 
(Weakly) Active Inequality Constraints: 0
Inactive Inequality Constraints: 0
Equality Constraints: 34
 
Variables: 42
(Weakly) Active Variable Bounds: 19
Fixed Variables: 4
 
Adding Jacobian elements for bounds... 2.944e-6 seconds
Assembling J_sparse... 5.508e-6 seconds
